## Exercise

## Download files

In [1]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2022-10-30 11:46:43--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: 'datasource.zip.1'

     0K ....                                                  100%  500M=0s

2022-10-30 11:46:45 (500 MB/s) - 'datasource.zip.1' saved [4249/4249]



## Set paths

In [2]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

## Import module

In [3]:
import pandas as pd
import glob
import xml.etree.ElementTree as ET
import datetime

## Extract
### CSV extract function

In [4]:
def extract_csv(file):
    df = pd.read_csv(file)
    return df

In [5]:
def extract_json(file):
    df = pd.read_json(file, lines=True)
    return df

In [6]:
def extract_xml(file):
    df = pd.DataFrame(['car_model','year_of_manufacture','price','fuel'])
    tree = ET.parse(file)
    root = tree.getroot()
    for car in root:
        car_model = car.find('car_model').text
        year = int(car.find('year_of_manufacture').text)
        price = float(car.find('price').text)
        fuel = car.find('fuel').text
        df.loc[len(df.index)] = {'car_model':car_model,'year_of_manufacture':year,'price':price,'fuel':fuel}
    return df

### Extract Function

In [7]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    for csvfile in glob.glob('datasource/*.csv'):
        extracted_data = pd.concat([extracted_data, extract_csv(csvfile)], ignore_index=True)
    
    for jsonfile in glob.glob('datasource/*.json'):
        extracted_data = pd.concat([extracted_data, extract_json(jsonfile)], ignore_index=True)
        
    for xmlfile in glob.glob('datasource/*.xml'):
        extracted_data = pd.concat([extracted_data, extract_xml(xmlfile)], ignore_index=True)
        
    return extracted_data

## Transform

transform price column into 2 decimal places

In [8]:
def transform(data):
    data['price'] = data['price'].apply(lambda x: float(f'{x:.2f}'))
    del data[0]
    data.dropna(axis=0, how='all', inplace=True)
    return data

## Loading 

In [9]:
def load(file, targetfile):
    file.to_csv(targetfile)

## Logging 

In [10]:
def log(message):
    timestamp_format = '%Y-%m-%d-%H:%M:%S'
    now = datetime.datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(logfile, 'a') as f:
        f.write(timestamp + ',' + message + '\n')

# ETL Process

In [11]:
log("ETL Job Started")

In [12]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,car_model,year_of_manufacture,price,fuel,0
0,ritz,2014,5000.000000,Petrol,NaN
1,sx4,2013,7089.552239,Diesel,NaN
2,ciaz,2017,10820.895522,Petrol,NaN
3,wagon r,2011,4253.731343,Petrol,NaN
4,swift,2014,6865.671642,Diesel,NaN
...,...,...,...,...,...
97,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN


In [13]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel
5,vitara brezza,2018,13805.97,Diesel
6,ciaz,2015,10074.63,Petrol
7,s cross,2015,9701.49,Diesel
8,ciaz,2016,13059.70,Diesel
9,ciaz,2015,11119.40,Diesel


In [14]:
log("Load phase Started")
load(transformed_data,targetfile)
log("Load phase Ended")